In [1]:
from scipy.optimize import minimize, curve_fit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
class Estagio:

  def __init__(self, armazenamentos, afluencias, carga, custo_otimo):

    self.armazenamentos = armazenamentos
    self.afluencias = afluencias
    self.carga = carga

    self.results = {}
    self.custo_otimo = custo_otimo
  

  def func_inter(self, x, a, b):
    
    return a*x + b

  def plot_interpolate(self):
    
    list_custos = []

    for ea in self.armazenamentos:
      custo = self.func_inter(ea, *self.model)
      list_custos.append(custo)

    plt.plot(self.armazenamentos, list_custos)
    plt.show()
  def interpolate_function(self, N_estagio):

      if N_estagio == 3:
      
        list_custo_otimo = np.zeros(len(self.armazenamentos))
      
      else:
        list_custo_otimo = []
        for key, value in self.custo_otimo.items():
          list_custo_otimo.append(value)

      model, _ = curve_fit(self.func_inter, xdata=self.armazenamentos, ydata=list_custo_otimo)

      return model

  def calculate_custo_imediato(self, gt1, gt2, defict):

    return 10 * gt1 + 25 * gt2 + 500 * defict

  def format_results(self, results):

    x = results['x']
    custo_imediato = self.calculate_custo_imediato(x[0], x[1], x[2])
    custo_futuro = self.func_inter(x[4], *self.model)

    variables = {}
    
    variables["gt1"] = x[0]
    variables["gt2"] = x[1]
    variables["deficit"] = x[2]
    variables["ghidrs"] = x[3]
    variables["ea"] = x[4]
    variables["evert"] = x[5]
    variables["custo-imediato"] = custo_imediato
    variables["custo-futuro"] = custo_futuro

    return variables

  def minimize(self, N_estagio):

    self.model = self.interpolate_function(N_estagio)

    for EA in self.armazenamentos:
      for EC in self.afluencias:

        ### X = ["gt1", "gt2", "deficit", "ghidrs", "ea", "evert"]
        function = lambda x: 10 * x[0] + 25 * x[1] + 500 * x[2] + 0.01 * x[5] \
                          + self.func_inter(EA + EC - x[3] - x[5], *self.model)
                          
        constraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] + x[2] + x[3] - self.carga},
                       {'type': 'eq', 'fun': lambda x: x[3] + x[4] + x[5] - EA - EC})
        
        limits = ((0, 15), (0, 10), (0, self.carga), (0, 76), (100, 160), (0, None))

        res = minimize(function, (1, 1, 1, 1, 1, 1), method='SLSQP', bounds=limits, constraints=constraints)

        self.results["EA"+str(EA)+"-EC"+str(EC)] = self.format_results(res)
        
    return self.results

  def calculate_custo_otimo(self):

    custo_otimo = {}
    for ea in self.armazenamentos:
      custo_ot = []

      for ec in self.afluencias:
        
        custo_ot.append(self.results["EA"+str(ea)+"-EC"+str(ec)]['custo-imediato'] + self.results["EA"+str(ea)+"-EC"+str(ec)]['custo-futuro'])

      custo_otimo[ea] = np.mean(custo_ot)

    return custo_otimo  


In [3]:
dict_vars = {3 : {"armazenamentos" : [160, 130, 100], "afluencias" : [42, 32], "carga":65},
             2:  {"armazenamentos" : [160, 130, 100], "afluencias" : [30, 18], "carga":65},
             1:  {"armazenamentos" : [160, 130, 100], "afluencias" : [26, 16], "carga":65}}

estagios = {}
for N_estagio, cenario in dict_vars.items():

  if N_estagio == 3:
    estagios[N_estagio] = Estagio(cenario['armazenamentos'], cenario['afluencias'], cenario['carga'], 0)

  else:
    estagios[N_estagio] = Estagio(cenario['armazenamentos'], cenario['afluencias'], cenario['carga'], estagios["custo_otimo-"+str(N_estagio + 1)])

  estagios["results-"+str(N_estagio)] = estagios[N_estagio].minimize(N_estagio)

  estagios["custo_otimo-"+str(N_estagio)] = estagios[N_estagio].calculate_custo_otimo()
  
  #estagios[N_estagio].plot_interpolate()

In [4]:
table_1 = pd.DataFrame(data=estagios["results-1"]).round()
table_2 = pd.DataFrame(data=estagios["results-2"]).round()
table_3 = pd.DataFrame(data=estagios["results-3"]).round()

In [5]:
table_1

,EA160-EC26,EA160-EC16,EA130-EC26,EA130-EC16,EA100-EC26,EA100-EC16
gt1,15.0,15.0,15.0,15.0,15.0,15.0
gt2,10.0,10.0,10.0,10.0,10.0,10.0
deficit,0.0,0.0,0.0,0.0,14.0,24.0
ghidrs,40.0,40.0,40.0,40.0,26.0,16.0
ea,146.0,136.0,116.0,106.0,100.0,100.0
evert,0.0,0.0,0.0,0.0,0.0,0.0
custo-imediato,400.0,400.0,400.0,400.0,7400.0,12400.0
custo-futuro,1688.0,3311.0,6559.0,8182.0,9156.0,9156.0


In [6]:
table_2

,EA160-EC30,EA160-EC18,EA130-EC30,EA130-EC18,EA100-EC30,EA100-EC18
gt1,15.0,15.0,15.0,15.0,15.0,15.0
gt2,10.0,10.0,10.0,10.0,10.0,10.0
deficit,0.0,0.0,0.0,0.0,10.0,22.0
ghidrs,40.0,40.0,40.0,40.0,30.0,18.0
ea,150.0,138.0,120.0,108.0,100.0,100.0
evert,0.0,0.0,0.0,0.0,0.0,0.0
custo-imediato,400.0,400.0,400.0,400.0,5400.0,11400.0
custo-futuro,5.0,480.0,1192.0,1667.0,1984.0,1984.0


In [7]:
table_3

,EA160-EC42,EA160-EC32,EA130-EC42,EA130-EC32,EA100-EC42,EA100-EC32
gt1,0.0,0.0,0.0,3.0,15.0,15.0
gt2,0.0,0.0,0.0,0.0,8.0,10.0
deficit,0.0,0.0,0.0,0.0,0.0,8.0
ghidrs,65.0,65.0,65.0,62.0,42.0,32.0
ea,137.0,127.0,107.0,100.0,100.0,100.0
evert,0.0,0.0,0.0,0.0,0.0,0.0
custo-imediato,0.0,0.0,0.0,30.0,350.0,4400.0
custo-futuro,0.0,0.0,0.0,0.0,0.0,0.0
